In [24]:
#spark setup
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NYC Taxi Big Data Task1") \
    .getOrCreate()


In [25]:
#load Dataset
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/content/yellow_tripdata_2015-01.csv")
df.printSchema()
df.count()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



8905520

In [23]:
#Remove Null Values
from pyspark.sql.functions import col

df_clean = df.dropna()
df_clean.count()

2483855

In [26]:
#Remove Invalid Records
df_clean = df_clean.filter(col("passenger_count") > 0)
df_clean = df_clean.filter(col("trip_distance") > 0)
df_clean = df_clean.filter(col("fare_amount") > 0)
df_clean = df_clean.filter(col("total_amount") > 0)


In [27]:
#Datetime Conversion
from pyspark.sql.functions import to_timestamp

df_clean = df_clean.withColumn(
    "tpep_pickup_datetime",
    to_timestamp("tpep_pickup_datetime")
).withColumn(
    "tpep_dropoff_datetime",
    to_timestamp("tpep_dropoff_datetime")
)


In [28]:
#Trip Duration Feature
from pyspark.sql.functions import unix_timestamp

df_clean = df_clean.withColumn(
    "trip_duration_minutes",
    (unix_timestamp("tpep_dropoff_datetime") -
     unix_timestamp("tpep_pickup_datetime")) / 60
)

df_clean = df_clean.filter(col("trip_duration_minutes") > 0)


In [29]:
#Cache
df_clean.cache()
df_clean.count()


2465692

In [30]:
                                 ##INSIGHTS##
#Peak Pickup Hours
from pyspark.sql.functions import hour

df_clean.withColumn(
    "pickup_hour", hour("tpep_pickup_datetime")
).groupBy("pickup_hour") \
 .count() \
 .orderBy("count", ascending=False) \
 .show()


+-----------+------+
|pickup_hour| count|
+-----------+------+
|         18|151430|
|         19|151184|
|         20|145681|
|         22|136836|
|         21|135255|
|         15|129883|
|         17|128006|
|         14|126259|
|         13|120737|
|         16|120607|
|         12|120291|
|         11|117822|
|          9|115131|
|         23|115118|
|         10|109812|
|          8|107363|
|          7| 92848|
|          0| 90486|
|          1| 68432|
|          2| 49091|
+-----------+------+
only showing top 20 rows


In [31]:
#Passenger Count
df_clean.groupBy("passenger_count") \
    .count() \
    .orderBy("passenger_count") \
    .show()


+---------------+-------+
|passenger_count|  count|
+---------------+-------+
|              1|1739793|
|              2| 351287|
|              3| 101906|
|              4|  48868|
|              5| 135564|
|              6|  88270|
|              7|      2|
|              8|      1|
|              9|      1|
+---------------+-------+



In [32]:
#Average Trip Distance per Hour
df_clean.withColumn(
    "pickup_hour", hour("tpep_pickup_datetime")
).groupBy("pickup_hour") \
 .avg("trip_distance") \
 .orderBy("pickup_hour") \
 .show()


+-----------+------------------+
|pickup_hour|avg(trip_distance)|
+-----------+------------------+
|          0| 3.272417280021217|
|          1| 3.657775748187991|
|          2|3.2752822309588363|
|          3| 3.461717343440816|
|          4| 4.116107124551519|
|          5| 4.564719938235877|
|          6|3.5860321015304195|
|          7|  2.85192982078235|
|          8| 2.450298519974288|
|          9| 3.239464523021616|
|         10|3.7823272502094354|
|         11|2.4693980750623825|
|         12| 2.479725582130001|
|         13|2.5742576840570743|
|         14|2.7182189784490607|
|         15|2.7764388719078035|
|         16|2.7884833384463765|
|         17|2.6158174616814924|
|         18| 5.798918180017162|
|         19|23.582917570642543|
+-----------+------------------+
only showing top 20 rows


In [33]:
#Revenue by Payment Type
df_clean.groupBy("payment_type") \
    .sum("total_amount") \
    .show()


+------------+--------------------+
|payment_type|   sum(total_amount)|
+------------+--------------------+
|           1|2.4898845879933514E7|
|           3|   77517.69999999771|
|           4|  24457.260000000166|
|           2|1.1366489439939272E7|
+------------+--------------------+



In [34]:
#Avg Fare vs Distance
df_clean.selectExpr(
    "avg(trip_distance) as avg_distance",
    "avg(total_amount) as avg_fare"
).show()


+-----------------+------------------+
|     avg_distance|          avg_fare|
+-----------------+------------------+
|4.519629750998842|14.749332146998151|
+-----------------+------------------+



In [35]:
df_clean.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/content/nyc_taxitrip_cleaned_dataset")

